## Initialize spark

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.4`

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

val spark = SparkSession
  .builder
  .master("local[2]")
  .getOrCreate()

import spark.implicits._

val sc = spark.sparkContext
sc.setLogLevel("WARN")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/11/15 08:26:09 INFO SparkContext: Running Spark version 2.4.4
19/11/15 08:26:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
19/11/15 08:26:09 INFO SparkContext: Submitted application: 8394513b-a063-4aed-83b0-986c9a9edbbf
19/11/15 08:26:09 INFO SecurityManager: Changing view acls to: yannis
19/11/15 08:26:09 INFO SecurityManager: Changing modify acls to: yannis
19/11/15 08:26:09 INFO SecurityManager: Changing view acls groups to: 
19/11/15 08:26:09 INFO SecurityManager: Changing modify acls groups to: 
19/11/15 08:26:09 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(yannis); groups with view permissions: Set(); users  with modify permissions: Set(yannis); groups with modify permissions: Set()
19/11/15 08:26:10 INFO Utils: Successfully started service 'sparkD

import $ivy.$                                  


import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


spark: SparkSession = org.apache.spark.sql.SparkSession@266004b6
import spark.implicits._


sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@1e27e18a

## Create a dataframe and transform it

In [2]:
val df = Seq(
  ("GR", "1", "10"),
  ("CY", "2", "20")
).toDF("COUNTRY", "R1", "R2")
  .withColumn("R1", concat_ws(":", lit("A"), 'R1))
  .withColumn("R2", concat_ws(":", lit("B"), 'R2))
  .select($"COUNTRY", explode(array("R1", "R2")).as("R"))
  .withColumn("SR", split('R, ":"))
  .withColumn("RR", 'SR(0))
  .withColumn("RRR", 'SR(1))
//   .select("COUNTRY", "RR", "RRR")
//   .groupBy("COUNTRY")
//   .pivot("RR")
//   .agg(first("RRR"))
//   .show

df: DataFrame = [COUNTRY: string, R: string ... 3 more fields]

### Just selecting and showing to appreciate the effects

In [3]:
df.show(false)

+-------+----+-------+---+---+
|COUNTRY|R   |SR     |RR |RRR|
+-------+----+-------+---+---+
|GR     |A:1 |[A, 1] |A  |1  |
|GR     |B:10|[B, 10]|B  |10 |
|CY     |A:2 |[A, 2] |A  |2  |
|CY     |B:20|[B, 20]|B  |20 |
+-------+----+-------+---+---+



In [4]:
df.select(first("COUNTRY")).show

+---------------------+
|first(COUNTRY, false)|
+---------------------+
|                   GR|
+---------------------+



In [5]:
df
  .groupBy("COUNTRY")
  .pivot("RR")
  .agg(first("RRR"))
  .show(false)

+-------+---+---+
|COUNTRY|A  |B  |
+-------+---+---+
|GR     |1  |10 |
|CY     |2  |20 |
+-------+---+---+



In [6]:
df
  .groupBy("COUNTRY")
  .pivot("RRR")
  .agg(first("RR"))
  .show(false)

+-------+----+----+----+----+
|COUNTRY|1   |10  |2   |20  |
+-------+----+----+----+----+
|GR     |A   |B   |null|null|
|CY     |null|null|A   |B   |
+-------+----+----+----+----+



## Expreriment with a new dataframe that includes JSON string

In [7]:
val json = """[ {"a": 1, "b": "two"}, {"b": "fourx", "a": 3} ]"""
val jsonDf = Seq(("first", json)).toDF("name", "value")

json: String = "[ {\"a\": 1, \"b\": \"two\"}, {\"b\": \"fourx\", \"a\": 3} ]"
jsonDf: DataFrame = [name: string, value: string]

In [8]:
val schema = ArrayType (
    StructType(
        StructField("a", IntegerType):: 
        StructField("b", StringType)::Nil
    )
)

// Spark>=2.4 way of defining a schema
val schema24 = schema_of_json(lit(jsonDf.select($"value").as[String].first))

val parsedJson = jsonDf
  .withColumn("parsed", from_json($"value", schema24))
  .withColumn("expl", explode($"parsed"))
  .drop("value","parsed")

schema: ArrayType = ArrayType(
  StructType(
    StructField("a", IntegerType, true, {}),
    StructField("b", StringType, true, {})
  ),
  true
)
schema24: Column = schemaofjson([ {"a": 1, "b": "two"}, {"b": "fourx", "a": 3} ])
parsedJson: DataFrame = [name: string, expl: struct<a: bigint, b: string>]

In [9]:
parsedJson.select("expl").show
parsedJson.select("expl.a").show
parsedJson.select("expl.b").show
parsedJson.select($"expl").as[String].first

+----------+
|      expl|
+----------+
|  [1, two]|
|[3, fourx]|
+----------+

+---+
|  a|
+---+
|  1|
|  3|
+---+

+-----+
|    b|
+-----+
|  two|
|fourx|
+-----+



res8_3: String = "[1, two]"